In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np

In [ ]:
from google.colab import files
a= files.upload()

Saving order_mun.geojson to order_mun.geojson


# Preprocess Odisseia data. Select indicators and tidy up dataframe

In [ ]:
#Select specific indices from Odisseia
# List of population values to filter
selected_id = [1064, 472, 1081, 19]

# Filter CSV data based on selected populations
filtered_csv_df = filtered[filtered['variavel_id'].isin(selected_id)]


In [ ]:
filtered_csv_df.to_csv('odisseia_sel.csv', index=False)

In [ ]:
#CHECK dtaframe from Odisseia
# Load CSV data
csv_path = "odisseia_sel.csv"
csv_df = pd.read_csv(csv_path, encoding='cp1252')
print(csv_df.info())
print(csv_df.head(5))

FileNotFoundError: ignored

In [ ]:
#Create new columns from column variables
csv_df
# Pivot the DataFrame to create new columns for each unique value in 'variavel_nome'
# Pivot the DataFrame using custom aggregation function
def custom_agg(x):
    return x.iloc[0]

pivot_df = csv_df.pivot_table(index=['municipio_codigo', 'tema_nome', 'ano', 'mes'],
                          columns='variavel_nome',
                          values=['valor'],
                          aggfunc=custom_agg).reset_index()

# Flatten the MultiIndex column labels
#pivot_df.columns = [f'{col[0]}_{col[1]}' if col[1] else col[0] for col in pivot_df.columns]
pivot_df.columns = ['municipio_codigo', 'Topic', 'Year', 'Month', 'Mortality', 'DAP Person', 'Amount of employed people', '% basic education']
#Change name
pivot_df.rename(columns={'municipio_codigo': 'CD_MUN'}, inplace=True)

# Display the pivot DataFrame
print(pivot_df.head(5))
print(pivot_df.info())

NameError: ignored

In [ ]:
# Select columns for comparison
column1 = mun['CD_MUN']
column2 = pivot_df['CD_MUN']

# Find matching values using isin() and count the matches
matching_count = column2.isin(column1).sum()

print("Number of matching codes:", matching_count)

Number of matching codes: 2112


In [ ]:
# Count the number of unique values in the selected column
unique_count = pivot_df['CD_MUN'].nunique()
print(unique_count)

66


In [ ]:
# Assuming your df DataFrame and geojson_df GeoDataFrame are defined
named_df = pivot_df.merge(mun[['CD_MUN', 'NM_MUN']], on='CD_MUN', how='left')
print(named_df.head(5))
print(named_df.info())

    CD_MUN       Topic  Year  Month  Mortality  DAP Person  \
0  2500304  EducaÃ§Ã£o  2016     12        NaN         NaN   
1  2500304  EducaÃ§Ã£o  2017     12        NaN         NaN   
2  2500304  EducaÃ§Ã£o  2018     12        NaN         NaN   
3  2500304  EducaÃ§Ã£o  2019     12        NaN         NaN   
4  2500304  EducaÃ§Ã£o  2020     12        NaN         NaN   

   Amount of employed people  % basic education         NM_MUN  
0                        NaN              73.75  Alagoa Grande  
1                        NaN              72.10  Alagoa Grande  
2                        NaN              70.35  Alagoa Grande  
3                        NaN              69.00  Alagoa Grande  
4                        NaN              96.05  Alagoa Grande  
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2112 entries, 0 to 2111
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   CD_MUN        

In [ ]:
named_df.to_csv('odisseia_col.csv', index=False)

# Load df that contains the 4 indicators from Odisseia

In [ ]:
#CHECK dtaframe from Odisseia
# Load CSV data
csv_path = "odisseia_col.csv"
csv_df = pd.read_csv(csv_path, encoding='cp1252')
print(csv_df.info())
print(csv_df.head(5))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2112 entries, 0 to 2111
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   CD_MUN                     2112 non-null   int64  
 1   Topic                      2112 non-null   object 
 2   Year                       2112 non-null   int64  
 3   Month                      2112 non-null   int64  
 4   Mortality                  660 non-null    float64
 5   DAP Person                 330 non-null    float64
 6   Amount of employed people  792 non-null    float64
 7   % basic education          330 non-null    float64
 8   NM_MUN                     2112 non-null   object 
dtypes: float64(4), int64(3), object(2)
memory usage: 148.6+ KB
None
    CD_MUN           Topic  Year  Month  Mortality  DAP Person  \
0  2500304  EducaÃƒÂ§ÃƒÂ£o  2016     12        NaN         NaN   
1  2500304  EducaÃƒÂ§ÃƒÂ£o  2017     12        NaN         NaN   
2  2500304 

Function to create a column of indicators

B The good option. Not used any longer

In [ ]:
def extract_indicators_values(row):
    indicators = ''
    values = np.nan
    columns = ['Mortality', 'DAP Person', 'Amount of employed people', '% basic education']

    for col in columns:
        if not np.isnan(row[col]):
            indicators = col
            values = row[col]
            break

    return indicators, values

# Apply the function row-wise to extract indicators and values
indicators_values = csv_df.apply(extract_indicators_values, axis=1, result_type='expand')

# Create new columns for 'indicator' and 'values'
csv_df['indicator'] = indicators_values[0]
csv_df['values'] = indicators_values[1]

# Convert 'values' column to float
csv_df['values'] = csv_df['values'].astype(float)

# Display the resulting DataFrame with the new columns
print(csv_df.info())
print(csv_df.head(5))
unique_indicators = csv_df['indicator'].drop_duplicates()

# Display the unique values in the 'indicator' column
print(unique_indicators)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2112 entries, 0 to 2111
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   CD_MUN                     2112 non-null   int64  
 1   Topic                      2112 non-null   object 
 2   Year                       2112 non-null   int64  
 3   Month                      2112 non-null   int64  
 4   Mortality                  660 non-null    float64
 5   DAP Person                 330 non-null    float64
 6   Amount of employed people  792 non-null    float64
 7   % basic education          330 non-null    float64
 8   NM_MUN                     2112 non-null   object 
 9   indicator                  2112 non-null   object 
 10  values                     2112 non-null   float64
dtypes: float64(5), int64(3), object(3)
memory usage: 181.6+ KB
None
    CD_MUN           Topic  Year  Month  Mortality  DAP Person  \
0  2500304  EducaÃƒÂ§ÃƒÂ£o  20

In [ ]:
# Drop specified columns
columns_to_drop = ['Topic', 'Year', 'Month', 'Mortality', 'DAP Person', 'Amount of employed people', '% basic education']
df_dropped = csv_df.drop(columns=columns_to_drop)

# Calculate percentage change for 'values' column based on 'indicator'
columns_to_calculate = ['values']

for col in columns_to_calculate:
    df_dropped[col + '_change'] = df_dropped.groupby('indicator')[col].pct_change() * 100

# Display the resulting DataFrame with percentage change columns
print(df_dropped)
print(df_dropped.info())

       CD_MUN         NM_MUN          indicator   values  values_change
0     2500304  Alagoa Grande  % basic education    73.75            NaN
1     2500304  Alagoa Grande  % basic education    72.10      -2.237288
2     2500304  Alagoa Grande  % basic education    70.35      -2.427184
3     2500304  Alagoa Grande  % basic education    69.00      -1.918977
4     2500304  Alagoa Grande  % basic education    96.05      39.202899
...       ...            ...                ...      ...            ...
2107  2517001      Umbuzeiro         DAP Person  1096.00      38.383838
2108  2517001      Umbuzeiro         DAP Person  1029.00      -6.113139
2109  2517001      Umbuzeiro         DAP Person   824.00     -19.922255
2110  2517001      Umbuzeiro         DAP Person   834.00       1.213592
2111  2517001      Umbuzeiro         DAP Person   717.00     -14.028777

[2112 rows x 5 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2112 entries, 0 to 2111
Data columns (total 5 columns):
 #   

Mean values for every time series (*provisional*)

In [ ]:
#Group by and calculate difference
mean_columns = ['Mortality', 'DAP Person', 'Amount of employed people', '% basic education']
grouped_mean = csv_df.groupby('CD_MUN')[mean_columns].mean().reset_index()

# Merge the grouped mean data with the original DataFrame using 'CD_MUN'
df_result = csv_df.merge(grouped_mean, on='CD_MUN', suffixes=('', '_mean'))

# Drop the original 4, 5, 6, 7, Year, and Month columns
columns_to_drop = ['Mortality', 'DAP Person', 'Amount of employed people', '% basic education', 'Year', 'Month','Topic']
df_result = df_result.drop(columns=columns_to_drop)

# Drop duplicate rows based on 'CD_MUN' and keep only the first occurrence
df_unique = df_result.drop_duplicates(subset='CD_MUN', keep ='first')
# Display the resulting DataFrame with unique rows
print(df_unique)
unique_nm_mun_count = df_unique['CD_MUN'].nunique()
print("Number of unique NM_MUN values:", unique_nm_mun_count)
# Display the resulting DataFrame with unique 'CD_MUN' values
print(df_unique)
print(df_result.info())
#Save averaged odissea data. No time series
df_unique.to_csv('od_mean.csv', index=False)

       CD_MUN         NM_MUN  Mortality_mean  DAP Person_mean  \
0     2500304  Alagoa Grande        102.9285           1374.2   
32    2500502      Alagoinha         90.2679            776.2   
64    2500601       Alhandra         48.3755            460.0   
96    2500809       AraÃ§agi         68.8277           1142.4   
128   2501104          Areia         83.4726            886.0   
...       ...            ...             ...              ...   
1952  2515906       Serraria         92.6940            414.4   
1984  2515930   SertÃ£ozinho        110.5726            296.8   
2016  2515971        Sobrado         60.1325            782.6   
2048  2516409         Tacima         77.5048            919.8   
2080  2517001      Umbuzeiro         62.5104            900.0   

      Amount of employed people_mean  % basic education_mean  
0                        1800.916667                   76.25  
32                        726.666667                   85.97  
64                       3236.

Adapted Geojson for the municipalities

In [ ]:
geojson_path = "order_mun.geojson"
mun = gpd.read_file(geojson_path)
print(mun)
print(mun.info())

     CD_MUN         NM_MUN SIGLA  AREA_KM2  \
0   2500304  Alagoa Grande    PB   322.071   
1   2500502      Alagoinha    PB   111.361   
2   2500601       Alhandra    PB   183.974   
3   2500809        Araçagi    PB   232.177   
4   2501104          Areia    PB   269.130   
..      ...            ...   ...       ...   
61  2515906       Serraria    PB    65.062   
62  2515930    Sertãozinho    PB    32.455   
63  2515971        Sobrado    PB    61.953   
64  2516409         Tacima    PB   245.236   
65  2517001      Umbuzeiro    PB   185.578   

                                             geometry  
0   MULTIPOLYGON (((-35.58723 -6.95548, -35.58732 ...  
1   MULTIPOLYGON (((-35.46875 -6.93963, -35.46756 ...  
2   MULTIPOLYGON (((-34.93716 -7.26991, -34.93784 ...  
3   MULTIPOLYGON (((-35.36915 -6.75595, -35.36924 ...  
4   MULTIPOLYGON (((-35.79893 -6.86625, -35.79882 ...  
..                                                ...  
61  MULTIPOLYGON (((-35.61821 -6.80927, -35.60694 ...  

Mean values for each indicator

In [ ]:
od_mean = pd.read_csv('od_mean.csv', encoding='cp1252')
# Display the resulting DataFrame with percentage change columns
print(od_mean)
print(od_mean.info())

     CD_MUN                   Topic          NM_MUN  Mortality_mean  \
0   2500304  EducaÃƒÆ’Ã‚Â§ÃƒÆ’Ã‚Â£o   Alagoa Grande        102.9285   
1   2500502  EducaÃƒÆ’Ã‚Â§ÃƒÆ’Ã‚Â£o       Alagoinha         90.2679   
2   2500601  EducaÃƒÆ’Ã‚Â§ÃƒÆ’Ã‚Â£o        Alhandra         48.3755   
3   2500809  EducaÃƒÆ’Ã‚Â§ÃƒÆ’Ã‚Â£o      AraÃƒÂ§agi         68.8277   
4   2501104  EducaÃƒÆ’Ã‚Â§ÃƒÆ’Ã‚Â£o           Areia         83.4726   
..      ...                     ...             ...             ...   
61  2515906  EducaÃƒÆ’Ã‚Â§ÃƒÆ’Ã‚Â£o        Serraria         92.6940   
62  2515930  EducaÃƒÆ’Ã‚Â§ÃƒÆ’Ã‚Â£o  SertÃƒÂ£ozinho        110.5726   
63  2515971  EducaÃƒÆ’Ã‚Â§ÃƒÆ’Ã‚Â£o         Sobrado         60.1325   
64  2516409  EducaÃƒÆ’Ã‚Â§ÃƒÆ’Ã‚Â£o          Tacima         77.5048   
65  2517001  EducaÃƒÆ’Ã‚Â§ÃƒÆ’Ã‚Â£o       Umbuzeiro         62.5104   

    DAP Person_mean  Amount of employed people_mean  % basic education_mean  
0            1374.2                     1800.916667                  

In [ ]:
# Drop columns 5, 6, and 7
columns_to_drop = ['Mortality', 'DAP Person', 'Amount of employed people']
df = csv_df.drop(columns=columns_to_drop)

# Filter rows with non-null values in the '% basic education' column
df = df[df['% basic education'].notnull()]

# Group by 'CD_MUN' and calculate the mean of '% basic education', keeping 'NM_MUN'
aggregated_df = df.groupby(['CD_MUN', 'NM_MUN'])['% basic education'].mean().reset_index()

# Display the aggregated DataFrame
print(aggregated_df)
print(aggregated_df.info())

aggregated_df.to_csv('odisseia_agg.csv', index=False)

     CD_MUN         NM_MUN  % basic education
0   2500304  Alagoa Grande              76.25
1   2500502      Alagoinha              85.97
2   2500601       Alhandra              85.08
3   2500809       AraÃ§agi              82.73
4   2501104          Areia              86.39
..      ...            ...                ...
61  2515906       Serraria              81.49
62  2515930   SertÃ£ozinho              87.15
63  2515971        Sobrado              75.54
64  2516409         Tacima              81.67
65  2517001      Umbuzeiro              91.23

[66 rows x 3 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66 entries, 0 to 65
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CD_MUN             66 non-null     int64  
 1   NM_MUN             66 non-null     object 
 2   % basic education  66 non-null     float64
dtypes: float64(1), int64(1), object(1)
memory usage: 1.7+ KB
None


In [ ]:

geojson_path = "bboxMunicipalities.geojson"
mun = gpd.read_file(geojson_path)
# Convert CD_MUN column to int64 data type
mun['CD_MUN'] = mun['CD_MUN'].astype('int64')
# Get the unique 'CD_MUN' values from the DataFrame
unique_cd_mun_values = aggregated_df['CD_MUN'].unique()
# Keep only the rows in geodataframe that have matching 'CD_MUN' values
m_gdf = mun[mun['CD_MUN'].isin(unique_cd_mun_values)]
# Merge the 'df' DataFrame with the 'geodataframe' to get the desired order
ordered_geodataframe = pd.merge(aggregated_df[['CD_MUN']], m_gdf, on='CD_MUN', how='left')

# Sort the ordered GeoJSON DataFrame based on the original order of 'CD_MUN' in 'df'
ordered_geodataframe.sort_values(by='CD_MUN', inplace=True)

# Display the ordered GeoJSON DataFrame
print(ordered_geodataframe)
print("\nGeoJSON DataFrame:")

print(ordered_geodataframe.info())


NameError: ignored

# Check structure of the municipalities Geodataframe

In [ ]:
import geopandas as gpd
# Specify the path and filename for the GeoJSON file
geojson_path = 'order_mun.geojson'
mun = gpd.read_file(geojson_path)
#geo_df = gpd.GeoDataFrame(output_geojson_path, geometry='geometry')
# Set 'CD_MUN' as the index
geo_df = mun.set_index('CD_MUN')
# Save the GeoDataFrame as a GeoJSON file
geo_df.to_file('order_mun', driver='GeoJSON')
# Display the ordered GeoJSON DataFrame
print(geo_df)
print("\nGeoJSON DataFrame:")

print(geo_df.info())

                NM_MUN SIGLA  AREA_KM2  \
CD_MUN                                   
2500304  Alagoa Grande    PB   322.071   
2500502      Alagoinha    PB   111.361   
2500601       Alhandra    PB   183.974   
2500809        Araçagi    PB   232.177   
2501104          Areia    PB   269.130   
...                ...   ...       ...   
2515906       Serraria    PB    65.062   
2515930    Sertãozinho    PB    32.455   
2515971        Sobrado    PB    61.953   
2516409         Tacima    PB   245.236   
2517001      Umbuzeiro    PB   185.578   

                                                  geometry  
CD_MUN                                                      
2500304  MULTIPOLYGON (((-35.58723 -6.95548, -35.58732 ...  
2500502  MULTIPOLYGON (((-35.46875 -6.93963, -35.46756 ...  
2500601  MULTIPOLYGON (((-34.93716 -7.26991, -34.93784 ...  
2500809  MULTIPOLYGON (((-35.36915 -6.75595, -35.36924 ...  
2501104  MULTIPOLYGON (((-35.79893 -6.86625, -35.79882 ...  
...                       